In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import os
import glob
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from pathlib import Path
from torch.optim.lr_scheduler import _LRScheduler, ReduceLROnPlateau

import json

from typing import List
import os
from functools import partial
import torch.nn.functional as F
from timeit import default_timer as timer
from utils import get_dataloaders

from models.utils import *
from datagen import kinematic_feature_names, kinematic_feature_names_jigsaws, kinematic_feature_names_jigsaws_patient_position, class_names, all_class_names, state_variables

from tqdm import tqdm

from collections import OrderedDict

from config import *

torch.manual_seed(0)



ImportError: cannot import name 'kinematic_feature_names_jigsaws_patient_position' from 'datagen' (/home/kesharaw/Desktop/repos/hamid/resilient-telesurgery/datagen.py)

Dataset & Dataloaders

In [ ]:

### -------------------------- DATA -----------------------------------------------------
tasks = ["Suturing"]
Features = kinematic_feature_names_jigsaws[38:] + state_variables #all patient side kinematic features + state variable features
# Features = kinematic_feature_names_jigsaws[0:]  #all patient side kinematic features + state variable features
# Features = kinematic_feature_names_jigsaws_patient_position + state_variables #kinematic features + state variable features
# Features = kinematic_feature_names_jigsaws_patient_position  #kinematic features only

one_hot = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
observation_window = 10
prediction_window = 10
batch_size = 64

user_left_out = 2
cast = True
include_image_features = False
normalizer = '' # ('standardization', 'min-max', 'power', '')
step = 1 # 1 - 30 Hz

train_dataloader, valid_dataloader = get_dataloaders(tasks,
                                                     user_left_out,
                                                     observation_window,
                                                     prediction_window,
                                                     batch_size,
                                                     one_hot,
                                                     class_names = class_names['Suturing'],
                                                     feature_names = Features,
                                                     include_image_features=include_image_features,
                                                     cast = cast,
                                                     normalizer = normalizer,
                                                     step=step)

print("datasets lengths: ", len(train_dataloader.dataset), len(valid_dataloader.dataset))
print("X shape: ", train_dataloader.dataset.X.shape, valid_dataloader.dataset.X.shape)
print("Y shape: ", train_dataloader.dataset.Y.shape, valid_dataloader.dataset.Y.shape)

# loader generator aragement: (src, tgt, future_gesture, future_kinematics)
print("Obs Kinematics Shape: ", train_dataloader.dataset[0][0].shape) 
print("Obs Target Shape: ", train_dataloader.dataset[0][1].shape)
print("Future Target Shape: ", train_dataloader.dataset[0][2].shape)
print("Future Kinematics Shape: ", train_dataloader.dataset[0][3].shape)
print("Train N Trials: ", train_dataloader.dataset.get_num_trials())
print("Train Max Length: ", train_dataloader.dataset.get_max_len())
print("Test N Trials: ", valid_dataloader.dataset.get_num_trials())
print("Test Max Length: ", valid_dataloader.dataset.get_max_len())
print("Features: ", train_dataloader.dataset.get_feature_names())



['G1' 'G10' 'G11' 'G2' 'G3' 'G4' 'G5' 'G6' 'G8' 'G9']
[0 0 0 ... 2 2 2]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
['G1' 'G10' 'G11' 'G2' 'G3' 'G4' 'G5' 'G6' 'G8' 'G9']
[0 0 0 ... 2 2 2]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
datasets lengths:  11080 1834
X shape:  (110801, 43) (18345, 43)
Y shape:  (110801, 10) (18345, 10)
Obs Kinematics Shape:  (10, 43)
Obs Target Shape:  (11, 10)
Future Target Shape:  (10, 10)
Future Kinematics Shape:  (10, 43)
Train N Trials:  34
Train Max Length:  9011
Test N Trials:  5
Test Max Length:  5555
Features:  ['PSML_position_x', 'PSML_position_y', 'PSML_position_z', 'PSML_rotation_0', 'PSML_rotation_1', 'PSML_rotation_2', 'PSML_rotation_3', 'PSML_rotation_4', 'PSML_rotation_5', 'PSML_rotation_6', 'PSML_rotation_7', 'PSML_rotati

In [ ]:

# Example tensor of shape [batch_size, seq_len, features]
tensor = torch.tensor([[[1, 2, 3], [3, 2, 1], [1, 1, 2]],
                       [[2, 3, 3], [1, 2, 2], [3, 2, 3]]])
tensor = torch.zeros(64,10,14)
print(tensor.shape)
# Calculate the most common feature along the seq_len axis for each batch
most_common_feature = torch.mode(tensor, dim=1).values

print(most_common_feature.shape)


torch.Size([64, 10, 14])
torch.Size([64, 14])


In [ ]:
def find_mostcommon(tensor,device):
    # # 64,10,10
    # # Count the occurrences of each row
    # unique_rows, counts = np.unique(batch_y, axis=0, return_counts=True)

    # # Get the index of the row with the highest count
    # most_common_row_index = np.argmax(counts)

    # # Get the most common row
    # most_common_row = unique_rows[most_common_row_index]
    
    # batch_y_bin = most_common_row
    
    # batch_y_bin = torch.from_numpy(batch_y_bin)
    
    batch_y_bin = torch.mode(tensor, dim=1).values
    batch_y_bin = batch_y_bin.to(device)
    
    return batch_y_bin
    

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):

        out, _ = self.lstm(x)

        out = self.fc(out)

        return out

In [ ]:

class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]



def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler



Transformer+TCN

In [ ]:

class GlobalMaxPooling1D(nn.Module):

    def __init__(self, data_format='channels_last'):
        super(GlobalMaxPooling1D, self).__init__()
        self.data_format = data_format
        self.step_axis = 1 if self.data_format == 'channels_last' else 2

    def forward(self, input):
        return torch.max(input, axis=self.step_axis).values
    

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)

        out = self.fc(out)

        return out
    
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        return hidden
    
class CNN_Encoder(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(CNN_Encoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=in_channels, out_channels=96, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride = 2),
            nn.Conv1d(in_channels=96, out_channels=128, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride = 2),
            nn.Conv1d(in_channels=128, out_channels=out_channels, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride = 2)
        )
    

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Reshape input to [batch_size, features, seq_len]
        x = self.encoder(x)
        # print('encoder_out',x.shape)
        return x
    
    
class CNN_Decoder(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(CNN_Decoder, self).__init__()

        self.decoder = nn.Sequential(
            nn.Upsample(scale_factor=2.0),
            nn.ConvTranspose1d(in_channels=in_channels, out_channels=96, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
            nn.ReLU(),
            nn.Upsample(scale_factor=2.0),
            nn.ConvTranspose1d(in_channels=96, out_channels=64, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
            nn.ReLU(),
            nn.Upsample(scale_factor=2.0),
            nn.ConvTranspose1d(in_channels=64, out_channels=out_channels, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
            nn.ReLU(),
        )
    def forward(self, x):
        x = self.decoder(x)
        # print('decoder_out',x.shape)
        return x
    
    
class TransformerModel(nn.Module):
    
    def __init__(self, input_dim, output_dim, d_model, nhead, num_layers, hidden_dim, layer_dim,dropout=0.1):
        super().__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout), num_layers=num_layers
        )
        
        self.lstm = LSTMModel(2*d_model, hidden_dim, layer_dim, int(2*d_model))
        
        self.gru = GRUNet(input_dim=2*d_model,hidden_dim=512,output_dim= output_dim,n_layers=4)
        
        encoder_paramss = { "in_channels": input_dim,
                    "kernel_size": 59,
                    "out_channels": 2*d_model,
                    }
        decoder_params = {
                    "in_channels": 2*d_model,
                    "kernel_size": 59,
                    "out_channels": int(d_model)
                        }
                                      
        self.encoder = CNN_Encoder(**encoder_paramss)
        self.decoder = CNN_Decoder(**decoder_params)
        
        self.max_pool = GlobalMaxPooling1D()
        self.fc = nn.Linear(input_dim, 2*d_model)
        # self.out = nn.Linear(int(d_model/2), output_dim)
        self.out = nn.Linear(d_model, output_dim) # vanilla + gru
        
    # tcn + transformer
    def forward(self, x):
        
        x = self.encoder(x)
        
        x = self.decoder(x)
        
        x = x.permute(0, 2, 1)  # Reshape input to [batch_size, seq_len,features, ]
        
        x = self.transformer(x)
        
        x = self.max_pool(x) # gets rid of seq_len

        x = self.out(x)
        
        return x
        

    


TCN

In [ ]:

class LSTM_Layer_tcn(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers,
                 bi_dir=True, use_gru=True):
        super(LSTM_Layer_tcn, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bi_dir = bi_dir
        self.use_gru = use_gru

        if self.use_gru:
            self.lstm = nn.GRU(input_size, hidden_size, num_layers, 
                                batch_first=True, bidirectional=bi_dir)
        else:
            self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                                batch_first=True, bidirectional=bi_dir)

    def forward(self, x): # x: (batch,feature,seq)

        x = x.permute(0, 2, 1) 

        batch_size = x.size(0)
        x, _ = self.lstm(x, self.__get_init_state(batch_size)) # x: (batch,seq,hidden)
        
        x = x.permute(0, 2, 1) 
        
        return x

    def __get_init_state(self, batch_size):

        if self.bi_dir:
            nl_x_nd = 2 * self.num_layers
        else:
            nl_x_nd = 1 * self.num_layers

        h0 = torch.zeros(nl_x_nd, batch_size, self.hidden_size)
        h0 = h0.cuda()

        if self.use_gru:
            return h0
        else:
            c0 = torch.zeros(nl_x_nd, batch_size, self.hidden_size)
            c0 = c0.cuda()
            return (h0, c0)


class ChannelNorm(nn.Module):
    def __init__(self):
        super(ChannelNorm, self).__init__()

    def forward(self, x): #(batch, feature, seq)
        divider = torch.max(torch.max(torch.abs(x), dim=0)[0], dim=1)[0] + 1e-5
        divider = divider.unsqueeze(0).unsqueeze(2)
        divider = divider.repeat(x.size(0), 1, x.size(2))
        x = x / divider
        return x


class Encoder(nn.Module):
    def __init__(self, input_size,
                 layer_type, layer_sizes, 
                 kernel_size=None, norm_type=None,
                 downsample=True):
        super(Encoder, self).__init__()

        if layer_type not in ['TempConv', 'Bi-LSTM']:
            raise Exception('Invalid Layer Type')
        if layer_type == 'TempConv' and kernel_size is None:
            raise Exception('Kernel Size For TempConv Not Specified')

        self.output_size = layer_sizes[-1]

        module_list = []

        for layer in range(len(layer_sizes)):
            if layer == 0:
                in_chl = input_size
            else:
                in_chl = layer_sizes[layer-1]
            out_chl = layer_sizes[layer]

            if layer_type == 'TempConv':
                conv_pad = kernel_size // 2
                module_list.append(('conv_{}'.format(layer), 
                    nn.Conv1d(in_chl, out_chl, kernel_size, padding=conv_pad)))
            elif layer_type == 'Bi-LSTM':
                module_list.append(('lstm_{}'.format(layer), 
                    LSTM_Layer_tcn(in_chl, out_chl // 2, 1, bi_dir=True)))

            if norm_type == 'Channel':
                module_list.append(('cn_{}'.format(layer), 
                    ChannelNorm()))
            elif norm_type == 'Batch':
                module_list.append(('bn_{}'.format(layer), 
                    nn.BatchNorm1d(out_chl)))
            elif norm_type == 'Instance':
                module_list.append(('in_{}'.format(layer), 
                    nn.InstanceNorm1d(out_chl)))
            else:
                print('No Norm Used!')           

            if layer_type == 'TempConv':
                module_list.append(('relu_{}'.format(layer), 
                    nn.ReLU()))
            else:
                pass

            if downsample:
                module_list.append(('pool_{}'.format(layer), 
                    nn.MaxPool1d(kernel_size=2, stride=2)))

        self.module = nn.Sequential(OrderedDict(module_list))


    def forward(self, x): # x: (batch,feature, seq)
        return self.module(x)



class Decoder(nn.Module):
    def __init__(self, input_size,
                 layer_type, layer_sizes, 
                 kernel_size=None, transposed_conv=None,
                 norm_type=None):
        super(Decoder, self).__init__()

        if layer_type not in ['TempConv', 'Bi-LSTM']:
            raise Exception('Invalid Layer Type')
        if layer_type == 'TempConv' and kernel_size is None:
            raise Exception('Kernel Size For TempConv Not Specified')
        if layer_type == 'TempConv' and transposed_conv is None:
            raise Exception('If Use Transposed Conv Not Specified')

        self.output_size = layer_sizes[-1]

        module_list = []

        for layer in range(len(layer_sizes)):
            if layer == 0:
                in_chl = input_size
            else:
                in_chl = layer_sizes[layer-1]
            out_chl = layer_sizes[layer]

            module_list.append(('up_{}'.format(layer), 
                nn.Upsample(scale_factor=2)))

            if layer_type == 'TempConv':
                conv_pad = kernel_size // 2
                if transposed_conv:
                    module_list.append(('conv_{}'.format(layer), 
                        nn.ConvTranspose1d(in_chl, out_chl, kernel_size, 
                                                    padding=conv_pad)))
                else:
                    module_list.append(('conv_{}'.format(layer), 
                        nn.Conv1d(in_chl, out_chl, kernel_size,
                                            padding=conv_pad)))
            elif layer_type == 'Bi-LSTM':
                module_list.append(('lstm_{}'.format(layer), 
                    LSTM_Layer_tcn(in_chl, out_chl // 2, 1, bi_dir=True)))

            if norm_type == 'Channel':
                module_list.append(('cn_{}'.format(layer), 
                    ChannelNorm()))
            elif norm_type == 'Batch':
                module_list.append(('bn_{}'.format(layer), 
                    nn.BatchNorm1d(out_chl)))
            elif norm_type == 'Instance':
                module_list.append(('in_{}'.format(layer), 
                    nn.InstanceNorm1d(out_chl)))
            else:
                print('No Norm Used!')           

            if layer_type == 'TempConv':
                module_list.append(('relu_{}'.format(layer), 
                    nn.ReLU()))
            else:
                pass

        self.module = nn.Sequential(OrderedDict(module_list))


    def forward(self, x): # x: (batch,feature, seq)
        return self.module(x)



class EncoderDecoderNet(nn.Module):
    def __init__(self, class_num, fc_size,
                 encoder_params, 
                 decoder_params, 
                 mid_lstm_params=None):

        super(EncoderDecoderNet, self).__init__()

        self.encoder = Encoder(**encoder_params)

        self.middle_lstm = None
        if mid_lstm_params is not None:
            self.middle_lstm = LSTM_Layer_tcn(mid_lstm_params['input_size'],
                                      mid_lstm_params['hidden_size'],
                                      mid_lstm_params['layer_num'],
                                      bi_dir=True)  # batch_first
        
        self.decoder = Decoder(**decoder_params)

        self.fc1 = nn.Linear(self.decoder.output_size, fc_size)
        self.fc2 = nn.Linear(fc_size, class_num)


    def forward(self, x):
        #x = x.permute(0, 2, 1) 
        

        x = F.relu(self.extract_feature(x))
        x = self.fc2(x)
        

        return x


    def extract_feature(self, x):

        x = x.permute(0, 2, 1)  

        x = self.encoder(x)
        if self.middle_lstm is not None:
            x = self.middle_lstm(x)

        x = self.decoder(x)
        x = x.permute(0, 2, 1)

        x = self.fc1(x)

        return x
    
    
    
class TCN(nn.Module):
    
    def __init__(self, input_dim, output_dim, tcn_model_params, dropout=0.1):
        super().__init__()

        tcn_model_params['encoder_params']['input_size'] = input_dim
        tcn_model_params['class_num'] = output_dim
        self.tcn = EncoderDecoderNet(output_dim,tcn_model_params['fc_size'],tcn_model_params['encoder_params'],tcn_model_params['decoder_params'],tcn_model_params['mid_lstm_params'])
        self.max_pool = GlobalMaxPooling1D()
        
    def forward(self, x):
        x = self.tcn(x)
  
        x=self.max_pool(x)

        return x 
    


In [ ]:
def initiate_model(input_dim, output_dim, transformer_params, learning_params , tcn_model_params, model):

    d_model,nhead, num_layers,hidden_dim,layer_dim = transformer_params.values()

    lr, epochs, weight_decay, patience, batch_size, seq_len = learning_params.values()

    if(model == 'transformer'):
        model = TransformerModel(input_dim=input_dim, output_dim=output_dim, d_model=d_model, nhead=nhead, num_layers=num_layers, hidden_dim=hidden_dim, layer_dim=layer_dim)

    elif(model == 'tcn'):
        model = TCN(input_dim=input_dim,output_dim=output_dim, tcn_model_params=tcn_model_params)
    
    model = model.cuda()

    # Define the optimizer (Adam optimizer with weight decay)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Define the learning rate scheduler (ReduceLROnPlateau scheduler)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=patience, verbose=True)

    criterion = nn.CrossEntropyLoss()
    
    return model, optimizer,scheduler, criterion


In [ ]:
def eval_loop(model, test_dataloader, criterion):
    model.eval()
    with torch.no_grad():
        # eval
        losses = []
        ypreds, gts = [], []
        
        for src, tgt, future_gesture, future_kinematics in test_dataloader:
            src = src
            tgt = tgt[:,1:,:]
            y = find_mostcommon(tgt, device)
            
            y_pred = model(src) # [64,10]
        
            pred = torch.argmax(y_pred,dim=-1)
            gt = torch.argmax(y,dim=1) # maxpool
            # gt = torch.argmax(tgt,dim=-1)
            
            pred = pred.cpu().numpy()
            gt = gt.cpu().numpy()
            
            ypreds.append(pred)
            gts.append(gt)
            
            loss = criterion(y_pred, y)

            losses.append(loss.item())
          
        ypreds = np.concatenate(ypreds)  
        gts = np.concatenate(gts)  
        
        # get_classification_report(ypreds,gts,test_dataloader.dataset.get_target_names())
        
        # Compare each element and count matches
        matches = np.sum(ypreds == gts)

        print('evaluation:',ypreds.shape, gts.shape, matches)

        # Calculate accuracy
        accuracy = matches / len(ypreds)

        print("Accuracy:", accuracy)
        
        return np.mean(losses),accuracy

In [ ]:

def traintest_loop(train_dataloader,test_dataloader,model,optimizer,scheduler,criterion, epochs):
    
    accuracy = 0
    total_accuracy = []
    # training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for bi, (src, tgt, future_gesture, future_kinematics) in enumerate(tqdm(train_dataloader)):

            optimizer.zero_grad()
            
            src = src
            y = find_mostcommon(tgt[:,1:,:], device)
            
            tgt = tgt[:,1:,:]
            
            y_pred = model(src) # [64,10]
            # print('input, prediction,gt:',src.shape, y_pred.shape,  y.shape, tgt.shape)
            
            loss = criterion(y_pred, y) #for maxpool
            # loss = criterion(y_pred, tgt)
            loss.backward()

            optimizer.step()
            
            running_loss += loss.item()
            
        # scheduler.step(running_loss)
        
        print(f"Training Epoch {epoch+1}, Loss: {running_loss / len(train_dataloader):.6f}")
        
        # evaluation loop
        val_loss, accuracy = eval_loop(model,test_dataloader,criterion)
        print(f"Valdiation Epoch {epoch+1}, Loss: {val_loss:.6f}")
        
        total_accuracy.append(accuracy)
        
    return val_loss,accuracy, total_accuracy

In [3]:
        

task = "Suturing"

epochs = learning_params["epochs"]
seq_len = learning_params["seq_len"]

iterations_per_epoch = 500

batch = next(iter(train_dataloader))
features = batch[0].shape[-1]
output_dim = batch[1].shape[-1]
input_dim = features  

print(input_dim, output_dim)


window_wise = True

# model = 'tcn' 
model = 'transformer'


model,optimizer,scheduler,criterion = initiate_model(input_dim=input_dim,output_dim=input_dim,transformer_params=transformer_params,learning_params=learning_params, tcn_model_params=tcn_model_params, model=model)

print(model)

NameError: name 'learning_params' is not defined

In [40]:
subjects = [2,3,4,5,6,7,8,9]
# subjects = [4]

observation_window = seq_len
with open("results.json", "w") as outfile:
    
    accuracy = []
    for subject in (subjects):

        model,optimizer,scheduler,criterion = initiate_model(input_dim=input_dim,output_dim=output_dim,transformer_params=transformer_params,learning_params=learning_params, tcn_model_params=tcn_model_params, model=model)
        user_left_out = subject

        train_dataloader, valid_dataloader = get_dataloaders(tasks,
                                                            user_left_out,
                                                            observation_window,
                                                            prediction_window,
                                                            batch_size,
                                                            one_hot,
                                                            class_names = class_names['Suturing'],
                                                            feature_names = Features,
                                                            include_image_features=include_image_features,
                                                            cast = cast,
                                                            normalizer = normalizer,
                                                            step=step,
                                                            window_wise= window_wise)



        val_loss,acc, total_acc = traintest_loop(train_dataloader,valid_dataloader,model,optimizer,scheduler,criterion, epochs)
        
        # train_loop(dataloader=train_dataloader, model=model, optimizer=optimizer, scheduler=scheduler, criterion=criterion, epochs=epochs)

        # # # evaluation loop
        # val_loss, acc = eval_loop(model,valid_dataloader,criterion)
        # print(f"Valdiation Loss: {val_loss:.6f}")
        
            
        accuracy.append({'subject':subject, 'last_accuracy':acc,  'avg_accuracy':np.mean(total_acc), 'highest_accuracy':np.max(total_acc)})

    # Serializing json
    json_object = json.dumps(accuracy, indent=4)
    outfile.write(json_object)
        


['G1' 'G10' 'G11' 'G2' 'G3' 'G4' 'G5' 'G6' 'G8' 'G9']
[0 0 0 ... 2 2 2]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
['G1' 'G10' 'G11' 'G2' 'G3' 'G4' 'G5' 'G6' 'G8' 'G9']
[0 0 0 ... 2 2 2]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


  0%|          | 0/55 [00:00<?, ?it/s]


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [55]:
val_loss, acc = eval_loop(model,valid_dataloader,criterion)
print(f"Valdiation Loss: {val_loss:.6f}")

        

Accuracy: 0.8254437869822485
Valdiation Loss: 0.740955


In [17]:
acc = []
for x in accuracy:
    print(x)
    acc.append(x['highest_accuracy'])
    
print(np.average(acc))

{'subject': 2, 'last_accuracy': 0.6983606557377049, 'avg_accuracy': 0.6784699453551912, 'highest_accuracy': 0.7278688524590164}
{'subject': 3, 'last_accuracy': 0.8458333333333333, 'avg_accuracy': 0.8284722222222222, 'highest_accuracy': 0.8708333333333333}
{'subject': 4, 'last_accuracy': 0.6825396825396826, 'avg_accuracy': 0.7407407407407407, 'highest_accuracy': 0.798941798941799}
{'subject': 5, 'last_accuracy': 0.8828451882845189, 'avg_accuracy': 0.8425383542538356, 'highest_accuracy': 0.8828451882845189}
{'subject': 6, 'last_accuracy': 0.8979591836734694, 'avg_accuracy': 0.9074829931972789, 'highest_accuracy': 0.9489795918367347}
{'subject': 7, 'last_accuracy': 0.7586206896551724, 'avg_accuracy': 0.74394341290893, 'highest_accuracy': 0.7957559681697612}
{'subject': 8, 'last_accuracy': 0.8674242424242424, 'avg_accuracy': 0.8762626262626263, 'highest_accuracy': 0.8977272727272727}
{'subject': 9, 'last_accuracy': 0.8284023668639053, 'avg_accuracy': 0.8496055226824457, 'highest_accuracy':

In [22]:
import optuna

# Define the objective function for Optuna
def objective(trial):
    # Hyperparameters to be tuned
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    d_model = 2**trial.suggest_int('d_model', 1, 8)
    nhead = 2**trial.suggest_int('nhead', 1, 8)
    num_layers = trial.suggest_categorical('num_layers', [1, 2, 4])
    hidden_dim = 2** trial.suggest_int('hidden_dim', 1, 8)
    layer_dim = trial.suggest_int('layer_dim', 1, 4)
    
    print(d_model, nhead)
    # Other hyperparameters
    # input_dim = 64  # Adjust according to your input data
    # output_dim = 10  # Adjust according to your output data
    val_loss = 1000
    if(d_model%nhead == 0):
        model,optimizer,scheduler,criterion = initiate_model(d_model, nhead, num_layers,input_dim,output_dim, hidden_dim, layer_dim, lr,  iterations_per_epoch=iterations_per_epoch, tcn_model_params=tcn_model_params, model=model)

        model.to(device)
        # Load your data and define loss function here
        
        train_dataloader, valid_dataloader = get_dataloaders(tasks,
                                                            user_left_out,
                                                            observation_window,
                                                            prediction_window,
                                                            batch_size,
                                                            one_hot,
                                                            class_names = class_names['Suturing'],
                                                            feature_names = Features,
                                                            include_image_features=include_image_features,
                                                            cast = cast,
                                                            normalizer = normalizer,
                                                            step=step)

        
        # Perform training loop with the defined batch size and model
        # ...

        val_loss = traintest_loop(train_dataloader,valid_dataloader,model,optimizer,scheduler,criterion, epochs)
        


    return val_loss  # Return the loss as the objective to minimize

if __name__ == '__main__':
    study = optuna.create_study(direction='minimize')  # Objective is to minimize the validation loss
    study.optimize(objective, n_trials=100)  # You can adjust the number of trials
    
    print(f"Number of finished trials: {len(study.trials)}")
    print("Best trial:")
    trial = study.best_trial

    print(f"Value: {trial.value}")
    print("Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

[I 2023-08-16 16:29:33,585] A new study created in memory with name: no-name-35d1af8d-1a39-42bf-b540-ff817f5cd8a2


128 32
[5555, 3420, 3267, 3256, 2847, 3155, 3266, 3048, 2449, 2527, 3583, 2925, 2516, 2869, 2482, 2803, 2374, 2235, 2372, 2028, 9011, 3378, 5160, 2620, 2453, 5274, 3320, 4020, 3279, 4310, 4411, 4145, 3730, 3700]
['G1' 'G10' 'G11' 'G2' 'G3' 'G4' 'G5' 'G6' 'G8' 'G9']
[0 0 0 ... 2 2 2]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
[2653, 2002, 2303, 2642, 1758]
['G1' 'G10' 'G11' 'G2' 'G3' 'G4' 'G5' 'G6' 'G8' 'G9']
[0 0 0 ... 2 2 2]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


100%|██████████| 184/184 [00:01<00:00, 96.72it/s] 


Training Epoch 1, Loss: 1.989756
Accuracy: 0.3185840707964602
Valdiation Epoch 1, Loss: 2.212640


100%|██████████| 184/184 [00:01<00:00, 101.36it/s]


...

Training Epoch 8, Loss: 1.681713
Accuracy: 0.44601769911504424
Valdiation Epoch 8, Loss: 1.822532


100%|██████████| 368/368 [00:01<00:00, 348.95it/s]


Training Epoch 9, Loss: 1.632332
Accuracy: 0.4407079646017699
Valdiation Epoch 9, Loss: 1.817733


100%|██████████| 368/368 [00:01<00:00, 325.96it/s]
[I 2023-08-16 16:45:37,943] Trial 99 finished with value: 1.7596882316801283 and parameters: {'lr': 5.529269206213509e-05, 'batch_size': 16, 'd_model': 8, 'nhead': 8, 'num_layers': 2, 'hidden_dim': 3, 'layer_dim': 3}. Best is trial 54 with value: 1.242251820034451.


Training Epoch 10, Loss: 1.578771
Accuracy: 0.4584070796460177
Valdiation Epoch 10, Loss: 1.759688
Number of finished trials: 100
Best trial:
Value: 1.242251820034451
Params: 
    lr: 8.906324028628413e-05
    batch_size: 64
    d_model: 8
    nhead: 8
    num_layers: 2
    hidden_dim: 4
    layer_dim: 1


In [26]:
for key, value in trial.params.items():
    print(f"    {key}: {value}")
    
print(f"Value: {trial.value}")

print(f"Number of finished trials: {len(study.trials)}")


    lr: 8.906324028628413e-05
    batch_size: 64
    d_model: 8
    nhead: 8
    num_layers: 2
    hidden_dim: 4
    layer_dim: 1
Value: 1.242251820034451
Number of finished trials: 100
